In [1]:
import numpy as np 
import cv2 as cv
import matplotlib.pyplot as plt 
import math 
import pandas as pd
import imutils.paths as path
from tqdm import tqdm

In [2]:
PATH = 'non-batik'
imagePaths = sorted(list(path.list_images(PATH)))
# print (imagePaths)

In [3]:
data = []
for i in tqdm(imagePaths,desc="load"):
    imgg=cv.imread (i)
    img = cv.cvtColor(imgg, cv.COLOR_BGR2GRAY)
    a = cv.resize (img, (128, 128))
    data.append (a)

load: 100%|██████████| 1089/1089 [00:08<00:00, 128.12it/s]


In [4]:
def derajat0 (img):
    max = np.max(img)
    imgTmp=np.zeros([max+1,max+1])
    for i in range (len(img)):
        for j in range (len(img[i])-1):
            imgTmp[img[i,j],img[i,j+1]] += 1
            
    data= imgTmp+np.transpose(imgTmp)
    tmp=np.sum(data)
            
    for i in range (len (data)):
        for j in range (len (data)):
            data[i,j]/=tmp
    return data

In [5]:
def derajat45 (img):
    max = np.max(img)
    imgTmp=np.zeros([max+1,max+1])
    for i in range (len (img)-1):
        for j in range (len (img[i])-1):
            imgTmp[img[i+1,j],img[i,j+1]] += 1
            
    data= imgTmp+np.transpose(imgTmp)
    tmp=np.sum(data)
    
    for i in range (len (data)):
        for j in range (len (data)):
            data[i,j]/=tmp
    return data

In [6]:
def derajat90 (img):
    max = np.max(img)
    imgTmp=np.zeros([max+1,max+1])
    for i in range (len (img)-1):
        for j in range (len (img[i])):
            imgTmp[img[i+1,j],img[i,j]] += 1
            
    data= imgTmp+np.transpose(imgTmp)
    tmp=np.sum(data)
            
    for i in range (len (data)):
        for j in range (len (data)):
            data[i,j]/=tmp
    return data

In [7]:
def derajat135 (img):
    max = np.max(img)
    imgTmp=np.zeros([max+1,max+1])
    for i in range (len (img)-1):
        for j in range (len (img[i])-1):
            imgTmp[img[i,j],img[i+1,j+1]] += 1
            
    data= imgTmp+np.transpose(imgTmp)
    tmp=np.sum(data)
            
    for i in range (len (data)):
        for j in range (len (data)):
            data[i,j]/=tmp
    return data

In [8]:
hasil=[]
for i in tqdm (range (len (data)),desc='glcm'):
    dat=[]
    dat.append(derajat0 (data[i]))
    dat.append(derajat45 (data[i]))
    dat.append(derajat90 (data[i]))
    dat.append(derajat135 (data[i]))
    hasil.append(dat)

glcm: 100%|██████████| 1089/1089 [02:30<00:00,  7.24it/s]


In [9]:
def contras (data):
    contras=0
    for i in range (len (data)):
        for j in range (len (data)):
            contras+=data[i,j]*pow(i-j,2)
    return contras

In [10]:
def entropy (data):
    entro=0
    for i in range (len (data)):
        for j in range (len (data)):
            if data[i,j] > 0.0:
                entro += -(data[i,j] * math.log(data[i,j]))
    return entro

In [11]:
def homogentitas (data):
    homogen=0
    for i in range (len (data)):
        for j in range (len (data)):
            homogen+=data[i,j]*(1+(pow(i-j,2)))
    return homogen

In [12]:
def energi (data):
    ener=0
    for i in range (len (data)):
        for j in range (len (data)):
            ener += data[i,j]**2
    return ener

In [13]:
data0=[]
x=['0','45','90','135']
hasilnya=[]
for j in tqdm (range(len(hasil)),desc="ektraksi"):
    da=[]
    da.append(imagePaths[j])
    for i in hasil[j]:
        dx=energi (i)
        da.append(dx)

        dh=homogentitas (i)
        da.append(dh)

        den=entropy (i)
        da.append(den)

        dco=contras (i)
        da.append(dco)
        
    hasilnya.append(da)

ektraksi: 100%|██████████| 1089/1089 [09:19<00:00,  1.95it/s]


In [15]:
headerssss=['path','energi0','homogenitas0','entropy0','contras0'
           ,'energi45','homogenitas45','entropy45','contras45'
           ,'energi90','homogenitas90','entropy90','contras90'
           ,'energi135','homogenitas135','entropy135','contras135']
df = pd.DataFrame(hasilnya, columns=headerssss)

In [16]:
df.head()
# df.shape

,path,energi0,homogenitas0,entropy0,contras0,energi45,homogenitas45,entropy45,contras45,energi90,homogenitas90,entropy90,contras90,energi135,homogenitas135,entropy135,contras135
0,non-batik\non (1).png,0.000854,51.749200,7.386075,50.749200,0.000673,86.240126,7.610928,85.240126,0.000725,73.974594,7.545667,72.974594,0.000638,95.775436,7.659017,94.775436
1,non-batik\non (10).png,0.000303,222.287586,8.441814,221.287586,0.000249,316.270135,8.605594,315.270135,0.000264,282.561885,8.559480,281.561885,0.000233,373.766384,8.665742,372.766384
2,non-batik\non (100).png,0.000372,193.849471,8.297116,192.849471,0.000317,265.358299,8.435949,264.358299,0.000436,142.776575,8.152621,141.776575,0.000321,271.197842,8.426993,270.197842
3,non-batik\non (1000).png,0.001687,47.362512,6.678087,46.362512,0.001208,113.730051,6.978991,112.730051,0.001421,69.299951,6.836270,68.299951,0.001758,49.953190,6.660978,48.953190
4,non-batik\non (1001).png,0.001268,59.546629,6.957278,58.546629,0.000875,129.990390,7.265694,128.990390,0.001135,70.227485,7.046787,69.227485,0.001404,56.391345,6.895138,55.391345


In [17]:
df

,path,energi0,homogenitas0,entropy0,contras0,energi45,homogenitas45,entropy45,contras45,energi90,homogenitas90,entropy90,contras90,energi135,homogenitas135,entropy135,contras135
0,non-batik\non (1).png,0.000854,51.749200,7.386075,50.749200,0.000673,86.240126,7.610928,85.240126,0.000725,73.974594,7.545667,72.974594,0.000638,95.775436,7.659017,94.775436
1,non-batik\non (10).png,0.000303,222.287586,8.441814,221.287586,0.000249,316.270135,8.605594,315.270135,0.000264,282.561885,8.559480,281.561885,0.000233,373.766384,8.665742,372.766384
2,non-batik\non (100).png,0.000372,193.849471,8.297116,192.849471,0.000317,265.358299,8.435949,264.358299,0.000436,142.776575,8.152621,141.776575,0.000321,271.197842,8.426993,270.197842
3,non-batik\non (1000).png,0.001687,47.362512,6.678087,46.362512,0.001208,113.730051,6.978991,112.730051,0.001421,69.299951,6.836270,68.299951,0.001758,49.953190,6.660978,48.953190
4,non-batik\non (1001).png,0.001268,59.546629,6.957278,58.546629,0.000875,129.990390,7.265694,128.990390,0.001135,70.227485,7.046787,69.227485,0.001404,56.391345,6.895138,55.391345
5,non-batik\non (1002).png,0.001322,43.762488,6.918369,42.762488,0.000817,115.767004,7.336788,114.767004,0.001050,68.773499,7.124991,67.773499,0.001344,54.792176,6.970747,53.792176
6,non-batik\non (1003).png,0.000130,685.764825,9.125413,684.764825,0.000128,764.644305,9.143838,763.644305,0.000131,649.498770,9.105792,648.498770,0.000110,1333.232996,9.287971,1332.232996
7,non-batik\non (1004).png,0.000130,583.637119,9.124417,582.637119,0.000116,835.567983,9.230481,834.567983,0.000112,914.938730,9.259465,913.938730,0.000101,1469.589745,9.368745,1468.589745
8,non-batik\non (1005).png,0.000107,747.106853,9.301254,746.106853,0.000104,871.181784,9.332824,870.181784,0.000109,707.266302,9.283108,706.266302,0.000090,1415.988282,9.472691,1414.988282
9,non-batik\non (1006).png,0.000115,639.022392,9.250658,638.022392,0.000102,872.543865,9.357407,871.543865,0.000097,1033.325664,9.398859,1032.325664,0.000085,1721.999628,9.530792,1720.999628


In [18]:
df.to_csv(r'non-batik.csv', index=False)